In [2]:
#pip install yfinance
#pip install PyPortfolioOpt

In [3]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pypfopt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

### Stock Selection

In [4]:
# Stock Selection
stock_list = ["D05.SI","Z74.SI","J36.SI","O39.SI","U11.SI","F34.SI","Y92.SI","Q0F.SI","BN4.SI","C38U.SI","C07.SI"]
#9CI.SI

In [5]:
START = '2015-01-01'
END = '2022-12-31'
df = pd.DataFrame()
period = "1y"
for company in stock_list:
    stock = yf.Ticker(company)
    hist = stock.history(start=START, end=END).Close
    df[company] = hist
df = df.dropna()
df

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
Date,,,,,,,,,,,
2015-01-02 00:00:00+08:00,13.822887,2.709047,45.657066,7.215527,17.721083,2.431841,0.002571,1.414135,6.068273,1.303257,30.024017
2015-01-05 00:00:00+08:00,13.512713,2.702082,45.305977,7.112447,17.323423,2.402093,0.002516,1.402575,5.951838,1.309645,29.306086
2015-01-06 00:00:00+08:00,13.371116,2.674225,44.693428,6.981880,16.860693,2.372346,0.002516,1.379456,5.698422,1.290480,28.645578
2015-01-07 00:00:00+08:00,13.350884,2.688154,44.327385,6.995625,16.947456,2.357473,0.002607,1.379456,5.718972,1.328811,28.523527
2015-01-08 00:00:00+08:00,13.539688,2.743867,44.484268,7.215527,17.366806,2.409531,0.002607,1.398722,5.801159,1.296868,28.645578
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+08:00,32.153217,2.490043,48.404598,11.612527,29.245789,3.981184,0.661024,1.662064,7.003848,1.916278,27.775372
2022-12-27 00:00:00+08:00,32.398808,2.499732,48.872322,11.574977,29.274185,3.981184,0.661024,1.670952,6.984816,1.916278,27.794725
2022-12-28 00:00:00+08:00,32.304348,2.499732,48.614597,11.593752,29.302576,4.038605,0.665849,1.653176,6.946752,1.916278,27.475357


### Weights Method

In [6]:
#Retriving mean using PyPortfolioOpt package
mu = mean_historical_return(df)
#Retriving covariance using PyPortfolioOpt package
S = CovarianceShrinkage(df).ledoit_wolf()

#Running efficient frontier function
ef = EfficientFrontier(mu, S)

#Get weights that maximises sharpe ratio, based of above efficient frontier
weights = ef.max_sharpe()

#What is our cut off for this do we need one? Also do we need to do this?
cleaned_weights = ef.clean_weights()
# print(cleaned_weights)

#Calculating portfolio performance metrics
ef.portfolio_performance(verbose=True)

Expected annual return: 48.8%
Annual volatility: 42.9%
Sharpe Ratio: 1.09


(0.48799361655630397, 0.4291054869091138, 1.0906260367987948)

In [7]:
#Visualizing the distribution of the different weights for the different stocks
stock_weights = list(cleaned_weights.values())
print(stock_weights)

[0.42802, 0.0, 0.0, 0.01122, 0.02957, 0.10248, 0.42871, 0.0, 0.0, 0.0, 0.0]


In [8]:
#Calculating the log returns from dataframe
log_returns = np.log(df/df.shift(1))
# log_returns

#Calculating normal returns from dataframe
# norm_returns = (df - df.shift(1)) / df.shift(1)
norm_returns = df.pct_change()
# print(norm_returns)

In [9]:
def calculate_annual_returns(weights, returns):
    # print(returns.mean())
    # print('----------------')
    # print(weights)
    return np.sum(returns.mean() * weights) * 252

#Annualised normal returns
yearly_log_returns = calculate_annual_returns(stock_weights, log_returns)

#Annualised normal returns
yearly_norm_returns = calculate_annual_returns(stock_weights, norm_returns)

print(yearly_log_returns)
print(yearly_norm_returns)

0.3524160540912235
0.5035928219612135


In [10]:
#Log convariance
log_returns_cov = log_returns.cov()

#Normal covariance and variance
norm_cov = norm_returns.cov()
norm_var = norm_returns.var()

In [11]:
def calculate_volatility(weights, returns_covariance):
    cov_dot_weights = np.dot(returns_covariance * 252, weights)
    variance = np.dot(weights, cov_dot_weights)
    return np.sqrt(variance)

#log volatility
log_volatility = calculate_volatility(stock_weights, log_returns_cov)
print(log_volatility)

#normal volatility
norm_volatility = calculate_volatility(stock_weights, norm_cov)
print(norm_volatility)

0.33418180123318275
0.4713658319250444


### TO DO CONVERT NORMAL STANDARD DEVIATION AND NORMAL SHARPE RATIO

Not sure if my formula is correct, can help me double check

In [12]:
risk_free_rate = 0

# Calculate the expected normal annualised portfolio return
log_portfolio_return = calculate_annual_returns(stock_weights, log_returns)
portfolio_return = np.exp(log_portfolio_return)-1

# Calculate the portfolio volatility
log_portfolio_volatility = calculate_volatility(stock_weights, log_returns_cov)
norm_portfolio_volatility = calculate_volatility(stock_weights, norm_cov)

# Calculate the Sharpe ratio
log_sharpe_ratio = (portfolio_return - risk_free_rate) / log_portfolio_volatility
norm_sharpe_ratio = (portfolio_return - risk_free_rate) / norm_portfolio_volatility

print("Portfolio Return:", portfolio_return)
print("Portfolio Volatility:", norm_portfolio_volatility)
print("Sharpe Ratio:", norm_sharpe_ratio)


Portfolio Return: 0.42250023766118394
Portfolio Volatility: 0.4713658319250444
Sharpe Ratio: 0.8963319125947361


### Post Processing Weights using DiscreteAllocation Library

In [13]:
portfolio_value = 1000000

# Calculate the number of shares to purchase using DiscreteAllocation
#latest_prices = get_latest_prices(df)
earliest_prices = df.iloc[0]

da = DiscreteAllocation(cleaned_weights, earliest_prices, total_portfolio_value=portfolio_value)
allocation, leftover = da.lp_portfolio(solver="ECOS_BB")

print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'D05.SI': 31841, 'Z74.SI': 20713, 'J36.SI': 208, 'O39.SI': 3257, 'U11.SI': 1744, 'F34.SI': 64504, 'Y92.SI': 8388608, 'Q0F.SI': 74690, 'BN4.SI': 2972, 'C38U.SI': 94057, 'C07.SI': 358}
Funds remaining: $4435.99


In [14]:
stock_shares_allocation = list(allocation.values())
print(stock_shares_allocation)

[31841, 20713, 208, 3257, 1744, 64504, 8388608, 74690, 2972, 94057, 358]


In [15]:
# Calculate the daily portfolio value
daily_portfolio_value = np.dot(df, stock_shares_allocation)
daily_portfolio_value += leftover
df_portfolio_value = pd.DataFrame(data={'Portfolio Value': daily_portfolio_value}, index=df.index)
portfolio_log_returns = np.log(df_portfolio_value/df_portfolio_value.shift(1))

# Annual Returns
portfolio_annual_returns = portfolio_log_returns.mean() * 252
annual_regular_ret = np.exp(portfolio_annual_returns)-1

# Standard Deviation
daily_portfolio_sd = (np.exp(portfolio_log_returns)-1).std()
annual_std =  daily_portfolio_sd * (252 **0.5)

# Sharpe Ratio
sharpe_ratio_discrete = (annual_regular_ret - risk_free_rate) / annual_std

print("Total Portfolio Return with Discrete Allocation:", annual_regular_ret)
print("Portfolio Volatility with Discrete Allocation:", annual_std)
print("Sharpe Ratio with Discrete Allocation:", sharpe_ratio_discrete)
# Calculate the daily portfolio return series
#daily_portfolio_return_series = daily_portfolio_value.pct_change().dropna()

Total Portfolio Return with Discrete Allocation: Portfolio Value    0.283514
dtype: float64
Portfolio Volatility with Discrete Allocation: Portfolio Value    0.355407
dtype: float64
Sharpe Ratio with Discrete Allocation: Portfolio Value    0.797716
dtype: float64


In [24]:
print(annual_regular_ret.values[0])
print(round(annual_regular_ret.values[0], 3))

0.2835135976473837
0.284


In [17]:
df_portfolio_value

,Portfolio Value
Date,
2015-01-02 00:00:00+08:00,1.000000e+06
2015-01-05 00:00:00+08:00,9.856307e+05
2015-01-06 00:00:00+08:00,9.727476e+05
2015-01-07 00:00:00+08:00,9.759453e+05
2015-01-08 00:00:00+08:00,9.866717e+05
...,...
2022-12-23 00:00:00+08:00,7.315705e+06
2022-12-27 00:00:00+08:00,7.324365e+06
2022-12-28 00:00:00+08:00,7.364038e+06


### Variation 1

### Sector Constraints

In [18]:
sector_mapper = {
    "D05.SI": "Finance",
    "Z74.SI": "Telecommunication",
    "J36.SI": "Industrials",
    "O39.SI": "Finance",
    "U11.SI": "Finance",
    "F34.SI": "Financials",
    "Y92.SI": "Consumer Defensive",
    "Q0F.SI": "Healthcare",
    "BN4.SI": "Industrials",
    "C38U.SI": "Real Estate",
    "C07.SI": "Industrials"
}
sector_lower = {"Industrials": 0.1, "Telecommunication": 0.1, "Healthcare": 0.05} 
sector_upper = {
    "Consumer Defensive": 0.3,
    "Finance": 0.4
}

In [30]:
mu1 = mean_historical_return(df)
S1 = CovarianceShrinkage(df).ledoit_wolf()

ef1 = EfficientFrontier(mu1, S1)

ef1.add_sector_constraints(sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper)

weights = ef1.max_sharpe()

cleaned_weights1 = ef1.clean_weights()
# print(cleaned_weights1)

ef1.portfolio_performance(verbose= True)


Expected annual return: 34.7%
Annual volatility: 31.3%
Sharpe Ratio: 1.05


(0.3471274521394521, 0.31272273857470423, 1.0460622519181055)